In [ ]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter
from torch.utils.data import DataLoader
from normalizer import getDataFromJSON, subSample, trainer, tester, getTotalAccuracy, cfvalidation
import torch.nn as nn
from skorch import NeuralNet

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load the Data ##

In [ ]:
# This file has 77270 data points. 
%time train1, labels1 = subSample(path="data/train_partition1_data.json", earlyStop=10000, device=device)

In [ ]:
# This file has 93767 data points. 
%time train2, labels2 = subSample(path="data/train_partition2_data.json", earlyStop=10000, device=device)

In [ ]:
# This file has 42986 data points. 
%time train3, labels3 = subSample(path="data/train_partition3_data.json", earlyStop=10000, device=device)

In [ ]:
train2[0,0]

## Define the network ##

This network has two parts - the stack and the fully connected layers. The stack interprets a given measurement and then outputs from 60 to a given number of entries. The fully connected part then makes a prediction off of all of that.

In [ ]:
# Define a network that takes each individual time series as input and outputs an nx33 tensor

class stack(nn.Module):
    def __init__(self, n):
        super().__init__() 
        self.n = n
        self.layer00 = nn.Linear(60,n)
        self.layer01 = nn.Linear(60,n)
        self.layer02 = nn.Linear(60,n)
        self.layer03 = nn.Linear(60,n)
        self.layer04 = nn.Linear(60,n)
        self.layer05 = nn.Linear(60,n)
        self.layer06 = nn.Linear(60,n)
        self.layer07 = nn.Linear(60,n)
        self.layer08 = nn.Linear(60,n)
        self.layer09 = nn.Linear(60,n)
        self.layer10 = nn.Linear(60,n)
        self.layer11 = nn.Linear(60,n)
        self.layer12 = nn.Linear(60,n)
        self.layer13 = nn.Linear(60,n)
        self.layer14 = nn.Linear(60,n)
        self.layer15 = nn.Linear(60,n)
        self.layer16 = nn.Linear(60,n)
        self.layer17 = nn.Linear(60,n)
        self.layer18 = nn.Linear(60,n)
        self.layer19 = nn.Linear(60,n)
        self.layer20 = nn.Linear(60,n)
        self.layer21 = nn.Linear(60,n)
        self.layer22 = nn.Linear(60,n)
        self.layer23 = nn.Linear(60,n)
        self.layer24 = nn.Linear(60,n)
        self.layer25 = nn.Linear(60,n)
        self.layer26 = nn.Linear(60,n)
        self.layer27 = nn.Linear(60,n)
        self.layer28 = nn.Linear(60,n)
        self.layer29 = nn.Linear(60,n)
        self.layer30 = nn.Linear(60,n)
        self.layer31 = nn.Linear(60,n)
        self.layer32 = nn.Linear(60,n)
        

    def forward(self, x):
        output = torch.zeros(x.shape[0], 33, self.n)
        output[:,0,:] = self.layer00(x[:,0,:])
        output[:,1,:] = self.layer01(x[:,1,:])
        output[:,2,:] = self.layer02(x[:,2,:])
        output[:,3,:] = self.layer03(x[:,3,:])
        output[:,4,:] = self.layer04(x[:,4,:])
        output[:,5,:] = self.layer05(x[:,5,:])
        output[:,6,:] = self.layer06(x[:,6,:])
        output[:,7,:] = self.layer07(x[:,7,:])
        output[:,8,:] = self.layer08(x[:,8,:])
        output[:,9,:] = self.layer09(x[:,9,:])
        
        output[:,10,:] = self.layer00(x[:,10,:])
        output[:,11,:] = self.layer00(x[:,11,:])
        output[:,12,:] = self.layer00(x[:,12,:])
        output[:,13,:] = self.layer00(x[:,13,:])
        output[:,14,:] = self.layer00(x[:,14,:])
        output[:,15,:] = self.layer00(x[:,15,:])
        output[:,16,:] = self.layer00(x[:,16,:])
        output[:,17,:] = self.layer00(x[:,17,:])
        output[:,18,:] = self.layer00(x[:,18,:])
        output[:,19,:] = self.layer00(x[:,19,:])
        
        output[:,20,:] = self.layer00(x[:,20,:])
        output[:,21,:] = self.layer00(x[:,21,:])
        output[:,22,:] = self.layer00(x[:,22,:])
        output[:,23,:] = self.layer00(x[:,23,:])
        output[:,24,:] = self.layer00(x[:,24,:])
        output[:,25,:] = self.layer00(x[:,25,:])
        output[:,26,:] = self.layer00(x[:,26,:])
        output[:,27,:] = self.layer00(x[:,27,:])
        output[:,28,:] = self.layer00(x[:,28,:])
        output[:,29,:] = self.layer00(x[:,29,:])
        
        output[:,30,:] = self.layer00(x[:,30,:])
        output[:,31,:] = self.layer00(x[:,31,:])
        output[:,32,:] = self.layer00(x[:,32,:])
        
        return output


In [ ]:
# Define the network. Make sure to end with nn.Softmax activation    
    
class CNNinspired(nn.Module):
    def __init__(self, n, hidden_size, num_classes=5, drop1=.5):
        super().__init__() 
        self.n = n
        self.layer1 = stack(n)
        self.layer2 = nn.Linear(n*33, hidden_size)
        self.layerout = nn.Linear(hidden_size, num_classes)
        #Define a RELU Activation unit
        self.relu = nn.ReLU()  
        self.sig = nn.Sigmoid()
        self.smax = nn.Softmax(dim=1)
        self.drop = nn.Dropout(p=drop1)
        self.b = True

    def forward(self, x):
        #Forward Propagate through the layers as defined above
        output = self.layer1(x)
        if self.b:
            self.out = output
            self.b = False
#         print(x.shape)
#         print('This is iffy')
#         print(output.shape)
#         assert False
        y = self.sig( output.reshape(-1, 33*self.n))
        y = self.relu(self.layer2(y))
        y = self.smax(self.layerout(y))
        return y 


## Begin testing with three fold validation ##

In [ ]:
modelArgs = [3, 128]
modelKwargs = {'drop1':.25}
trainKwargs = {'epochs':50, 'lr' : 0.01}
# Attempted learing rates: 
# 0.00001 - one in a million change
# 0.000001 - one in 10 million
# 0.0001 - one in 100 thousand
# 0.001 with 128 hidden neurons instead of 30 - one in 10 thousand

    # model1 = CNNinspired(1, 30)
    # model3 = CNNinspired(3, 30)

In [ ]:
train1Args = [CNNinspired, torch.cat((train1, train2), dim=0), labels1 + labels2, None, train3, labels3, None]
%time train12 = trainer(*train1Args, *modelArgs, **trainKwargs, **modelKwargs)

In [ ]:
train13 = trainer(
    CNNinspired,
    torch.cat((train1, train3), dim=0),
    labels1 + labels3,
    None,
    train2,
    labels2,
    None,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

In [ ]:
train23 = trainer(
    CNNinspired,
    torch.cat((train3, train2), dim=0),
    labels3 + labels2,
    None,
    train1,
    labels1,
    None,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

In [ ]:
acc = getTotalAccuracy(train12, train23, train13, 
                       torch.cat((train1, train2, train3), dim=0),
                       labels1 + labels2 + labels3)
print('Average accuracy of all three models on all data: ', acc)

In [ ]:
# def cfvalidation(tripList, module, x1, y1, x2, y2, x3, y3, **trainKwargs):
#         for (modelArgs, modelKwargs, lr) in tripList:
ns = [1,3,5,7]
hls = [30, 128, 256]
margs = []
for n in ns:
    for h in hls:
        margs.append([n,h])
lrs = [10**i for i in range(0,-5, -1)]
drs = [0, .05, .25, .45, .5]
tripList = []
for ma in margs:
    for r in lrs:
        for d in drs:
            tripList.append([ma, {'drop1': d}, r])
lossArgs, quickArgs = cfvalidation(tripList, CNNinspired, train1, labels1, train2, labels2, train3, labels3, 200)
print(lossArgs, quickArgs, sep='\n')

## Final model creation for testing ##

Once acceptable hyperparameters have been established, run this code to train on all the data and print out a CSV that predicts from the test data.

In [1]:
train123 = trainer(
    CNNinspired,
    torch.cat((train1, train2, train3), dim=0),
    labels1 + labels2 + labels3,
    None,
    torch.cat((train1, train2, train3), dim=0),
    labels1 + labels2 + labels3,
    None,
    epochs=1,
    *modelArgs,
    **trainKwargs,
    **modelKwargs
)

NameError: name 'trainer' is not defined

In [ ]:
# from datetime import datetime
# def tester(model, pathToWrite=None):
#     if pathToWrite is None:
#         pathToWrite = f'results/submission{datetime.now().strftime("%d_%H:%M")}.csv'
#     # Get test data
#     test, ids, _ = getDataFromJSON(path='data/test_4_5_data.json', test=True, device=device)
#     # get our guesses from the network
#     guesses = torch.argmax(model(test))
#     assert len(ids) == guesses.shape
#     # Open a file to write to
#     file = open(pathToWrite, mode='w')
#     print('Id,Label', file=file)
#     for i in range(len(ids)):
#         print(ids[i], guesses[i], sep=',', file=file)
#     file.close()

tester(train123)

In [ ]:
from datetime import datetime
acc = 
PATH = f'savedModels/lr{lr}acc{acc}time{datetime.now().strftime("%d_%H:%M")}.pth'
torch.save(newModel.state_dict(), PATH)
print('REMEMBER TO DELETE YOUR ACCURACY SO THE NEXT PERSON REMEMBERS TO WRITE THEIRS')